In [6]:
from rdflib import Graph
from typing import Optional
from pydantic import BaseModel
from api.src.function.loc.getType import GetType

In [54]:
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element
class Agents(BaseModel):
    type: str 
    # adminMetadata: AdminMetadata 
    elementList: list[Element]
    fullerName: Optional[Element]
    # note: Optional[str]
    # hasReciprocalAuthority: Optional[list[Uri]]
    # hasReciprocalExternalAuthority: Optional[list[Uri]]
    # hasBroaderAuthority: Optional[list[Uri]]
    # hasBroaderExternalAuthority: Optional[list[Uri]]
    # hasNarrowerAuthority: Optional[list[Uri]]
    # hasNarrowerExternalAuthority: Optional[list[Uri]]
    # hasCloseExternalAuthority: Optional[list[Uri]]
    # hasExactExternalAuthority: Optional[list[Uri]]
    # hasVariant: Optional[list[Variant]]
    # subjectOf: Optional[list[Uri]]
    # isMemberOfMADSCollection: str

In [15]:
authority = 'http://id.loc.gov/authorities/names/n80002329'
graph = Graph()
graph.parse(f'{authority}.madsrdf.rdf')

<Graph identifier=N0c3d1fbdee144e1781c329f2d6d0dd75 (<class 'rdflib.graph.Graph'>)>

In [67]:
def getElementList(graph, authority, obj):
    
  qElementList = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?type ?value 
    WHERE  {{
    <{authority}> madsrdf:elementList ?elementList .
    ?elementList rdf:rest* ?rest .
    ?rest rdf:first ?first .
    ?first rdf:type ?type  .
    ?first madsrdf:elementValue ?value
      }}"""
  r = graph.query(qElementList)
  bindings = r.bindings
  elementList = list()

  for i in bindings:
    element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  i.get('value').toPython(),
            "lang": i.get('lang')
          }
        }
    elementList.append(element)
  obj['elementList'] = elementList

  return obj

def getFullerName(graph, authority, obj):
    
  qFullerName = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?type ?value 
    WHERE  {{
    <{authority}> madsrdf:fullerName ?fullerName .
      ?fullerName rdf:type ?type .
      ?fullerName rdfs:label ?value
      }}"""
  r = graph.query(qFullerName)
  bindings = r.bindings
  if len(bindings) > 0:
    result = bindings[0]
    element = {
              "type": result.get('type').split("#")[1],
            "elementValue": {
              "value": result.get('value').toPython(),
              "lang": result.get('lang')
            }
          }
    obj['fullerName'] = element
    
  return obj

def GetExternalUri(authority, graph, metadata, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:has{metadata}Authority ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
            url = i.get('value')
            base = url.split("//")[1].split("/")[0]
            uri = {
                   "value": url,
                   "base": base,
                   "label": {
                       "value": i.get('label').value,
                       "lang": i.get('label').language } }
            external.append(uri)
        return external
    else:
        return False

In [68]:
hasCloseExternal = GetExternalUri(authority, graph, "hasCloseExternalAuthority", {})
hasCloseExternal 

False

In [64]:
def ParserAgents(graph, authority):
    # Type
    tipo = GetType(graph, authority)

    # adminMetadata
    adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]}
    
    obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'}
    
    # ElementList
    obj = getElementList(graph, authority, obj)
    print(obj)
    
    # fullerName
    obj = getFullerName(graph, authority, obj)
    # hasCloseExternalAuthority

    
    return obj



obj = ParserAgents(graph, authority)
obj

{'type': 'PersonalName', 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc', 'identifiedBy': [{'type': 'Lccn', 'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc', 'value': 'n80002329'}]}, 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/PersonalName/', 'elementList': [{'type': 'FullNameElement', 'elementValue': {'value': 'Machado de Assis,', 'lang': None}}, {'type': 'DateNameElement', 'elementValue': {'value': '1839-1908', 'lang': None}}]}


{'type': 'PersonalName',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'n80002329'}]},
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/PersonalName/',
 'elementList': [{'type': 'FullNameElement',
   'elementValue': {'value': 'Machado de Assis,', 'lang': None}},
  {'type': 'DateNameElement',
   'elementValue': {'value': '1839-1908', 'lang': None}}],
 'fullerName': {'type': 'PersonalName',
  'elementValue': {'value': 'Joaquim Maria Machado', 'lang': None}}}

In [65]:
request = Agents(**obj)
request

Agents(type='PersonalName', elementList=[Element(type='FullNameElement', elementValue=Label(value='Machado de Assis,', lang=None)), Element(type='DateNameElement', elementValue=Label(value='1839-1908', lang=None))], fullerName=Element(type='PersonalName', elementValue=Label(value='Joaquim Maria Machado', lang=None)))

In [36]:
element = {}
for i in bindings:
    tipo = i.get('type').split('#')[-1]
    element[tipo] = i.get('value').toPython()
    

element

{'FullNameElement': 'Machado de Assis,', 'DateNameElement': '1839-1908'}